# Using NLP Cleaning Techniques & Creating Price-Reference DF

First, I import my NLP packages

In [115]:
import pandas as pd
import regex as re
import numpy as np
import string
import pickle
import statistics

import nltk
nltk.download('wordnet')
nltk.download('words')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.text import Text

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/caitlinsanderson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/caitlinsanderson/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/caitlinsanderson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [235]:
 with open("title_pickle_str" + ".txt","rb") as f:
        title_word_dump = pickle.load(f)

In [236]:
title_word_dump

'polo golf   corduroy velvet check jacket  ysl yves saint laurent short sleeve tshirt size s m walter van beirendonck aw  hand on heart sweater new archive  tracksuit jacket nike vintage puffer down jacket fleece supreme long sleeve mike kelley big logo young thug x hm thugger hoodie rare s nudie jeans thin finn back  black black men jeans  the north face face  down vest l brand new long moon woman tshirt travis scott style burberry embossed metallic bronze leather notebook white warm vintage longsleeve shirt arcteryx polartec fleece jacket m size vintage evisu custom made denim jeans  nwt carhartt wip active jacket soft teal size m derby county home   adult shirt umbro jersey top  destroy bondage muslin vintage carhartt active work jacket yeezy slide bone wmns keb trousers size  marcelo burlon wings print tshirt size m dunk high shirt jacket overshirt  a bathing ape logo hoodie till black friday crazy deals shirt denim embroidery aap rocky striped tee wool knit roll neck sweater size 

I notice there are some strange spacing issues, I will clean the string with my clean_text function before tokenizing it. 

In [237]:
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) #take out anything in brackets
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #remove punctuation
    text = re.sub('^\$', '', text) #get rid of the $ before the prices
    text = re.sub('[‘’“”…\']', '', text) #get rid of more gobbledygook
    text = text.strip()
    text = " ".join(text.split())
    return text

cleaner = lambda x: clean_text(x)

In [238]:
clean_text(title_word_dump)

'polo golf corduroy velvet check jacket ysl yves saint laurent short sleeve tshirt size s m walter van beirendonck aw hand on heart sweater new archive tracksuit jacket nike vintage puffer down jacket fleece supreme long sleeve mike kelley big logo young thug x hm thugger hoodie rare s nudie jeans thin finn back black black men jeans the north face face down vest l brand new long moon woman tshirt travis scott style burberry embossed metallic bronze leather notebook white warm vintage longsleeve shirt arcteryx polartec fleece jacket m size vintage evisu custom made denim jeans nwt carhartt wip active jacket soft teal size m derby county home adult shirt umbro jersey top destroy bondage muslin vintage carhartt active work jacket yeezy slide bone wmns keb trousers size marcelo burlon wings print tshirt size m dunk high shirt jacket overshirt a bathing ape logo hoodie till black friday crazy deals shirt denim embroidery aap rocky striped tee wool knit roll neck sweater size it m l slim gr

## Using NLP techniques to prep my data

First I tokenize my title text

In [239]:
title_tokens = word_tokenize(title_word_dump)
print(title_tokens)

['polo', 'golf', 'corduroy', 'velvet', 'check', 'jacket', 'ysl', 'yves', 'saint', 'laurent', 'short', 'sleeve', 'tshirt', 'size', 's', 'm', 'walter', 'van', 'beirendonck', 'aw', 'hand', 'on', 'heart', 'sweater', 'new', 'archive', 'tracksuit', 'jacket', 'nike', 'vintage', 'puffer', 'down', 'jacket', 'fleece', 'supreme', 'long', 'sleeve', 'mike', 'kelley', 'big', 'logo', 'young', 'thug', 'x', 'hm', 'thugger', 'hoodie', 'rare', 's', 'nudie', 'jeans', 'thin', 'finn', 'back', 'black', 'black', 'men', 'jeans', 'the', 'north', 'face', 'face', 'down', 'vest', 'l', 'brand', 'new', 'long', 'moon', 'woman', 'tshirt', 'travis', 'scott', 'style', 'burberry', 'embossed', 'metallic', 'bronze', 'leather', 'notebook', 'white', 'warm', 'vintage', 'longsleeve', 'shirt', 'arcteryx', 'polartec', 'fleece', 'jacket', 'm', 'size', 'vintage', 'evisu', 'custom', 'made', 'denim', 'jeans', 'nwt', 'carhartt', 'wip', 'active', 'jacket', 'soft', 'teal', 'size', 'm', 'derby', 'county', 'home', 'adult', 'shirt', 'umbr

Now I will add the brand list I created from the top 50 brands to my stopwords list to filter them out from the text.

In [240]:
with open("strip_brand_list","rb") as f:
    strip_brand_list = pickle.load(f)

In [241]:
strip_brand_string = " ".join(strip_brand_list)
strip_brand_tokens = word_tokenize(strip_brand_string)
strip_brand_tokens

['nike',
 'adidas',
 'vintage',
 'gucci',
 'burberry',
 'stone',
 'island',
 'prada',
 'supreme',
 'maison',
 'margiela',
 'carhartt',
 'acne',
 'studios',
 'rick',
 'owens',
 'saint',
 'laurent',
 'paris',
 'moncler',
 'jordan',
 'brand',
 'luxury',
 'levis',
 'polo',
 'ralph',
 'lauren',
 'custom',
 'dolce',
 'gabbana',
 'dior',
 'the',
 'north',
 'face',
 'american',
 'vintage',
 'offwhite',
 'dries',
 'van',
 'noten',
 'bape',
 'balenciaga',
 'flannel',
 'arcteryx',
 'vivienne',
 'westwood',
 'versace',
 'louis',
 'vuitton',
 'streetwear',
 'evisu',
 'rick',
 'owens',
 'drkshdw',
 'comme',
 'des',
 'garcons',
 'our',
 'legacy',
 'designer',
 'yves',
 'saint',
 'laurent',
 'palm',
 'angels',
 'raf',
 'simons',
 'patagonia',
 'ysl',
 'pour',
 'homme',
 'lacoste',
 'helmut',
 'lang',
 'barbour',
 'japanese',
 'brand',
 'kith']

In [242]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(strip_brand_tokens)

I will import a list of colors from wikipedia to filter those out as well

In [243]:
url = ('https://simple.wikipedia.org/wiki/List_of_colors')

In [244]:
color_df = pd.read_html(url)
color_df[0]

,Name,Color,HTML (HEX) Code,Red (RGB) (0 - 255),Green (RGB) (0 - 255),Blue (RGB) (0 - 255),Hue (HSL) (0 - 360),Sat (HSL) (0 - 100),Lum (HSL) (0 - 10)
0,Amaranth,NaN,#E52B50,229,43,80,348*,81%,53%
1,Amber,NaN,#FFBF00,255,191,0,45*,100%,50%
2,Amethyst,NaN,#9966CC,153,102,204,270*,50%,60%
3,Apricot,NaN,#FBCEB1,251,206,177,24*,29%,84%
4,Aquamarine,NaN,#7FFFD4,127,255,212,160*,50%,75%
...,...,...,...,...,...,...,...,...,...
82,Ultramarine,NaN,#3F00FF,63,0,255,255*,100%,50%
83,Violet,NaN,#7F00FF,127,0,255,270*,100%,50%
84,Viridian,NaN,#40826D,64,130,109,161*,51%,38%
85,White,NaN,#FFFFFF,255,255,255,any,any,100%


I want only the first column

In [245]:
colornames = [color for color in color_df[0]['Name']]

colornames

['Amaranth',
 'Amber',
 'Amethyst',
 'Apricot',
 'Aquamarine',
 'Azure',
 'Baby blue',
 'Beige',
 'Brick red',
 'Black',
 'Blue',
 'Blue-green',
 'Blue-violet',
 'Blush',
 'Bronze',
 'Brown',
 'Burgundy',
 'Byzantium',
 'Carmine',
 'Cerise',
 'Cerulean',
 'Champagne',
 'Chartreuse green',
 'Chocolate',
 'Cobalt blue',
 'Coffee',
 'Copper',
 'Coral',
 'Crimson',
 'Cyan',
 'Desert sand',
 'Electric blue',
 'Emerald',
 'Erin',
 'Gold',
 'Gray',
 'Green',
 'Harlequin',
 'Indigo',
 'Ivory',
 'Jade',
 'Jungle green',
 'Lavender',
 'Lemon',
 'Lilac',
 'Lime',
 'Magenta',
 'Magenta rose',
 'Maroon',
 'Mauve',
 'Navy blue',
 'Ochre',
 'Olive',
 'Orange',
 'Orange-red',
 'Orchid',
 'Peach',
 'Pear',
 'Periwinkle',
 'Persian blue',
 'Pink',
 'Plum',
 'Prussian blue',
 'Puce',
 'Purple',
 'Raspberry',
 'Red',
 'Red-violet',
 'Rose',
 'Ruby',
 'Salmon',
 'Sangria',
 'Sapphire',
 'Scarlet',
 'Silver',
 'Slate gray',
 'Spring bud',
 'Spring green',
 'Tan',
 'Taupe',
 'Teal',
 'Turquoise',
 'Ultramari

In [246]:
strip_color_string = " ".join(colornames)
strip_colors = clean_text(strip_color_string)

In [247]:
strip_color_tokens = word_tokenize(strip_colors)
strip_color_tokens

['amaranth',
 'amber',
 'amethyst',
 'apricot',
 'aquamarine',
 'azure',
 'baby',
 'blue',
 'beige',
 'brick',
 'red',
 'black',
 'blue',
 'bluegreen',
 'blueviolet',
 'blush',
 'bronze',
 'brown',
 'burgundy',
 'byzantium',
 'carmine',
 'cerise',
 'cerulean',
 'champagne',
 'chartreuse',
 'green',
 'chocolate',
 'cobalt',
 'blue',
 'coffee',
 'copper',
 'coral',
 'crimson',
 'cyan',
 'desert',
 'sand',
 'electric',
 'blue',
 'emerald',
 'erin',
 'gold',
 'gray',
 'green',
 'harlequin',
 'indigo',
 'ivory',
 'jade',
 'jungle',
 'green',
 'lavender',
 'lemon',
 'lilac',
 'lime',
 'magenta',
 'magenta',
 'rose',
 'maroon',
 'mauve',
 'navy',
 'blue',
 'ochre',
 'olive',
 'orange',
 'orangered',
 'orchid',
 'peach',
 'pear',
 'periwinkle',
 'persian',
 'blue',
 'pink',
 'plum',
 'prussian',
 'blue',
 'puce',
 'purple',
 'raspberry',
 'red',
 'redviolet',
 'rose',
 'ruby',
 'salmon',
 'sangria',
 'sapphire',
 'scarlet',
 'silver',
 'slate',
 'gray',
 'spring',
 'bud',
 'spring',
 'green',


In [248]:
stopwords.extend(strip_color_tokens)

Turning now to fabric words...

In [249]:
#copied from: https://sewguide.com/fabric-glossary/
fabrics= """Abaca
Aba
Acetate 
Acrylic fabric
Active Comfort Denim
Admiralty cloth
Aerophane
Aertex
Aida Canvas / Aida cloth
Airplane cloth
Albert Cloth
Albatross
Alepine
Alpaca
Alpaca crepe
American Pima Cotton
Angora fibres
Angola
Anti-pill
Antique satin
 Ardass
Argyle
Armure
Art Linen
Astrakhan
Atlas
Awnings
Baby Combing Wool
Baft
Bagheera Velvet
Ballistic
Balloon cloth
Banbury plush
Bamboo fibers
Banana Fabric
Bandana
Baize
Bark Cloth
Baronet satin
Basket weave
Bathroom Blanketing (Blanket cloth)
Batik
Batiste
Batt or Batting
Beaded
Beaver cloth
Belly Wool
Bemberg
Bengaline /Faille
Binding cloth
Blanket cloth
Bobbinet
Bombazine
Bunting
Burlap
Bedford (Cord)
Berber fleece
Bicast leather (PU leather)
Biopolished cotton
Black Wool
Blends
Bonded leather
Bombazine
Boiled Wool
Botany Wools
Boynge
Breech or Britch Wool
Brocade
Broderie Anglaise
Brocatelle
Brushed Wool
Buck Fleece
Burlap
Calico
Cambric
Camel’s Hair
Camlet
Canton crepe
Canvas
Casement
Cashmere
Carpet Wool
Cashgora
Cashmerlon
Cerecloth / altar cloth
Cavalry Twill
Challis
Chambray / Chambric
Charmeuse
Chamoise
Charvet silk
Chanel tweed
Checks
Cheese cloth
Chenille
Cheviot
Chevron
Chiffon
China silk 
Chinchilla cloth
Chint
Chino
Chintz
Chite
Chire
Cisele velvet
Cloque
Coating
Colorfast
Combed cotton
Corduroy
Cottagora
Cotton
Cotton backed Satin
Cotton voile
Cotton Lawn
Crazy Horse
Crochet
Crepe
Crepe back satin
Cretonne
Crepe de chine
Crinkle satin
Crinoline 
Crinoline net
Cupro
Dacron
Duplex prints
Damask
Dazzle
Deerskin
Denim / dungaree / jean
Diaper cloth
Dimity
Doeskin
Dommet flannel
Donegal
Dorian
Dotted Swiss
Double cloth
Double Gauze
Double knit
Down
Dress net
Drill
Duchess satin
Duck cloth
Duvetyne
Duffel
Dupioni
Egyptian cotton
Epyngle
Eyelet
Eyelash
Elastane
Elastique
Eolienne
English net
Faconne
Faille
Fake fur
Feathers
Faux leather
Felt
Flannel
Flanellette
Flax
Foulard
French terry
Frieze
Full-grain leather
Faux Fur
Faux silk
Frieze
Fustian
Fur
Gabardine
Gauze
Georgette
Gingham
Glen checks
Gossamer
Grosgrain
Handkerchief Linen
Habutai
Haircloth
Hessian
Hemp
Homespun
Hopsacking
Huckaback
Ikat
Illusion
Interlock
Irish poplin
Jaconet
Jacquard
Jersey
Jute
Khakhi
Kidskin
Knit
Types of knit
Lace
Lamb’s wool
Lambskin
Layette
Linen
Different linen fabric
Linsey woolsey
Lint
Lisle
Llama
Loden Fabric
Lycra
Lurex
Leather
Leathertte
Liquid cotton
Lycra
Macrame
Macinaw
Mackinosh
Madras
Marled
Marquissette
Marvello
Marceline
Matelasse
Melange
Merino  wool
Messaline
Mesh
Melton
Microfiber
Milanese
Milk yarn
Modal
Moire
Mohair
Moleskin
Monks cloth
Moss crepe
Mother of pearl
Mull
Muslin
Nankeen
Nappa Leather
Neoprene
Nep
Netting
Net fabric
Ninon
Nonwoven fabric
Nubuck
Nylon
Nytril
Oilcloth
Olefin
Organdie / Organdy
Organic cotton
Organza
Ottoman rib
Oilcloth
Outing Flannel
Oxford cloth
Paisley
Panama cloth
Panne
Pashmina
Patent Leather
Peached fabric
Percale
Performance Knit
Permanent press fabric
Peau de Soie
Petersham
Pile knit
Pile weave
Pill
Pilot cloth
Pima cotton
Pincord
Pinpoint
Pina Fabric (Pineapple)
Pinstripe
Pique
Plisse
Plush
Point d’esprit
Poiret
Pongee silk
Poodle cloth
Polo cloth
Polished cotton
Polyester
Polyethylene
Polypropylene
Polyresin
Polystyrene
Ponte Roma
Poplin
Poult de soi
Quilted fabric
Rabbit hair/wool
Raccoon fur
Radium
Raffia
Ramie
Raschel knit
Rayon
Rayon Spandex
Repp
Resin
Rib Knit
Ribbon
Ric Rac
Ringspun fabric
Ripstop
Russet
Sailcloth
Santoprene
Sarcenet
Sarong skirt
Sateen
Satin
Types of satin and satin weave
Seersucker
Scrim
Serge
Serpentine crepe
Sharkskin
Shantung
Sheeting
Sherpa (fleece)
Silesie
Silk Satin
Silk
Types of silk fabric and silk weaves
Simplex
Sinamay
Sisal
Slipper Satin
Slub jersey
Spandex
Stitch bonded fabric
Stone washed
Surah
Suede
Suedecloth
Sueded fleece
Supima
Supriva
Swiss Dot
Synthetic
Sweater knit
Tactel
Taffeta
Tapa cloth
Tape yarn
Tapestry
Tarpaulin
Tartan
Tattersall
Teflon
Terrycloth
Terry Velvet
Thai silk
Thermal knit
Ticking
Tissue
Toweling
Toile
Transparent Velvet
Tropical wool
Tricot
Tricotine
Tri Acetate
Tricollete
Tsumugi Silk
Tufted fabric
Tulle
Tusseh silk / Tussah silk
Tweed
Twill
Ultrasuede
Velboa
Veloutine
Velour
Velvet
Velveteen
Velveteen plush
Venecia
Venetian fabric 
Venice
Vichy
Vicuna
Vinyl
Viscose
Voile
Washable Paper
Wadmal
Waffle cloth
Whipcord
Wincey
Wirecloth
Wool
Fabrics of wool fabric and weave
Wool crepe
Woolsy
Worsted wool
Worsted 
Worcester
Yak
Yoryu
Zanella
Zephyr
Zibeline"
"""

In [250]:
strip_fabrics = clean_text(fabrics)

In [251]:
strip_fabric_tokens = word_tokenize(strip_fabrics)
strip_fabric_tokens

['abaca',
 'aba',
 'acetate',
 'acrylic',
 'fabric',
 'active',
 'comfort',
 'denim',
 'admiralty',
 'cloth',
 'aerophane',
 'aertex',
 'aida',
 'canvas',
 'aida',
 'cloth',
 'airplane',
 'cloth',
 'albert',
 'cloth',
 'albatross',
 'alepine',
 'alpaca',
 'alpaca',
 'crepe',
 'american',
 'pima',
 'cotton',
 'angora',
 'fibres',
 'angola',
 'antipill',
 'antique',
 'satin',
 'ardass',
 'argyle',
 'armure',
 'art',
 'linen',
 'astrakhan',
 'atlas',
 'awnings',
 'baby',
 'combing',
 'wool',
 'baft',
 'bagheera',
 'velvet',
 'ballistic',
 'balloon',
 'cloth',
 'banbury',
 'plush',
 'bamboo',
 'fibers',
 'banana',
 'fabric',
 'bandana',
 'baize',
 'bark',
 'cloth',
 'baronet',
 'satin',
 'basket',
 'weave',
 'bathroom',
 'blanketing',
 'blanket',
 'cloth',
 'batik',
 'batiste',
 'batt',
 'or',
 'batting',
 'beaded',
 'beaver',
 'cloth',
 'belly',
 'wool',
 'bemberg',
 'bengaline',
 'faille',
 'binding',
 'cloth',
 'blanket',
 'cloth',
 'bobbinet',
 'bombazine',
 'bunting',
 'burlap',
 'bed

In [252]:
stopwords.extend(strip_fabric_tokens)

In [253]:
title_tokens_without_sw = [word for word in title_tokens if not word in stopwords]

print(title_tokens_without_sw)

['golf', 'check', 'jacket', 'short', 'sleeve', 'tshirt', 'size', 'walter', 'beirendonck', 'aw', 'hand', 'heart', 'new', 'archive', 'tracksuit', 'jacket', 'puffer', 'jacket', 'long', 'sleeve', 'mike', 'kelley', 'big', 'logo', 'young', 'thug', 'x', 'hm', 'thugger', 'hoodie', 'rare', 'nudie', 'jeans', 'thin', 'finn', 'men', 'jeans', 'vest', 'l', 'new', 'long', 'moon', 'woman', 'tshirt', 'travis', 'scott', 'style', 'embossed', 'metallic', 'notebook', 'warm', 'longsleeve', 'shirt', 'polartec', 'jacket', 'size', 'made', 'jeans', 'nwt', 'wip', 'jacket', 'soft', 'size', 'derby', 'county', 'home', 'adult', 'shirt', 'umbro', 'top', 'destroy', 'bondage', 'work', 'jacket', 'yeezy', 'slide', 'bone', 'wmns', 'keb', 'trousers', 'size', 'marcelo', 'burlon', 'wings', 'print', 'tshirt', 'size', 'dunk', 'high', 'shirt', 'jacket', 'overshirt', 'bathing', 'ape', 'logo', 'hoodie', 'till', 'friday', 'deals', 'shirt', 'embroidery', 'aap', 'rocky', 'striped', 'tee', 'roll', 'neck', 'size', 'l', 'slim', 'grey',

In [254]:
title_tokens_cln = Text(title_tokens_without_sw)
title_tokens_cln

<Text: golf check jacket short sleeve tshirt size walter...>

In [255]:
fdisttitle = FreqDist(title_tokens_cln)
fdisttitle.most_common(100)

[('jacket', 1160),
 ('shirt', 803),
 ('size', 680),
 ('logo', 637),
 ('tshirt', 528),
 ('hoodie', 512),
 ('x', 505),
 ('pants', 486),
 ('jeans', 480),
 ('tee', 466),
 ('sweatshirt', 386),
 ('rare', 365),
 ('big', 242),
 ('l', 231),
 ('zip', 222),
 ('new', 220),
 ('coat', 182),
 ('mens', 168),
 ('sleeve', 166),
 ('long', 165),
 ('bag', 159),
 ('air', 157),
 ('travis', 148),
 ('cargo', 147),
 ('bomber', 143),
 ('xl', 141),
 ('hat', 139),
 ('style', 130),
 ('made', 129),
 ('grey', 128),
 ('print', 126),
 ('scott', 124),
 ('sneakers', 121),
 ('shorts', 120),
 ('beanie', 114),
 ('high', 113),
 ('puffer', 112),
 ('track', 110),
 ('boots', 107),
 ('retro', 106),
 ('low', 105),
 ('crewneck', 102),
 ('cap', 100),
 ('check', 97),
 ('vest', 95),
 ('last', 95),
 ('top', 93),
 ('swoosh', 93),
 ('scarf', 91),
 ('light', 89),
 ('monogram', 87),
 ('belt', 85),
 ('drop', 81),
 ('necklace', 81),
 ('dunk', 80),
 ('trousers', 79),
 ('nova', 76),
 ('short', 74),
 ('longsleeve', 74),
 ('yeezy', 73),
 ('stri

There is clearly more cleaning that could be done in order to bring up words that are used for items, but I will work with this for now, along with words from my own experience that are used.  <br><br>
For purposes of this POC, I will manually create my own dictionary that associates certain words with our own, internal type categories.  <br><br>
First, I will bring in our type categories - what we call 'type groups'

In [256]:
bst_type_df = pd.read_excel("/Users/caitlinsanderson/Documents/ironhack_course_work/final project/1213bst_typegroups.xlsx")
bst_type_df

,ID,NAME
0,1,Tops
1,2,Dresses
2,3,Pants
3,4,Rings
4,5,Skirts
5,6,Full Body
6,7,Jackets
7,8,Blazers
8,9,Footwear
9,10,Accessories


In [257]:
with open("typegroups","wb") as f:
    pickle.dump(bst_type_df, f)

I will use our type_id, which is also the primary key for this table in our postgresql database, as the keys in my dictionary. <br><br>
Since this proof of concept is happening only with Grailed, which is a mens clothing marketplace, I will not spend time populating the type categories for 'dress' or 'skirt'.  There will likely be some products I will miss, but for now, I won't worry about that. 

In [258]:
bst_category_dic = {'top':1,
                    'shirt':1,
                    'tshirt':1,
                    'hoodie':1,
                    'tee':1,
                    'sweater':1,
                    'sweatshirt':1,
                    'vest':1,
                    'jumper':1, 
                    'cardigan':1, 
                    'pullover':1, 
                    'tank':1, 
                    'turtleneck':1, 
                    'dress':2, 
                    'pants':3, 
                    'trousers':3, 
                    'jeans':3, 
                    'shorts':3, 
                    'sweatpants':3, 
                    'jogging':3, 
                    'ring':4, 
                    'skirt':5,
                    'full body':6,
                    'romper':6,
                    'jumpsuit':6,
                    'suit':6,
                    'tracksuit':6,
                    'bodysuit':6,
                    'leotard':6,
                    'jacket':7,
                    'puffer':7,
                    'coat':7, 
                    'bomber':7,
                    'parka':7,
                    'anorak':7,
                    'raincoat':7,
                    'windbreaker':7,
                    'trench':7,
                    'blazer':8,
                    'shoe':9,
                    'sneakers':9,
                    'sneaker':9,
                    'high-tops':9, 
                    'high tops':9, 
                    'boots':9,
                    'loafers':9,
                    'derby':9, 
                    'boots':9,
                    'trainer':9,
                    'accessory':10,
                    'belt':10,
                    'scarf':10,
                    'socks':10,
                    'gloves':10,
                    'tie':10,
                    'cravat':10,
                    'handkerchief':10,
                    'bag':11,
                    'tote':11,
                    'duffel':11,
                    'duffle':11,
                    'briefcase':11,
                    'attache':11,
                    'case':11,
                    'pack':11,
                    'backpack':11,
                    'fanny':11,
                    'wallet':11,
                    'jewelry':12, 
                    'necklace':12,
                    'bracelet':12,
                    'pendant':12,
                    'earring':12,
                    'earrings':12,
                    'hat':13,
                    'cap':13,
                    'beanie':13,
                    'bowler':13,
                    'glasses':14,
                    'sunglasses':14,
                    'frames':14,
                    'bathing suit':15,
                    'trunks':15,
                    'bikini':15,
                    'merchandise':16,
                    'keychain':16,
                    'misc':17,
                    'boxers':17,
                    'briefs':17}

In [259]:
with open("clean_df_1","rb") as f:
    clean_df = pickle.load(f)

clean_df.head()

,brand1,brand2,brand3
0,harrington,polo ralph lauren,vintage
1,ysl pour homme,yves saint laurent,nan
2,walter van beirendonck,nan,nan
3,massimo osti,nan,nan
4,nike,vintage,nan


In [260]:
with open("cleaner_df","rb") as f:
    cleaner_df = pickle.load(f)
    
cleaner_df.head()

,brand,title,sold_price,when_sold,item_links
0,harrington × polo ralph lauren × vintage,polo golf corduroy velvet check jacket,72.8,2020-12-16 22:02:51.900669,/listings/16896292-harrington-x-polo-ralph-lau...
1,ysl pour homme × yves saint laurent,ysl yves saint laurent short sleeve tshirt siz...,48.0,2020-12-16 22:01:51.974796,/listings/18764205-ysl-pour-homme-x-yves-saint...
2,walter van beirendonck,walter van beirendonck aw hand on heart sweat...,132.0,2020-12-16 21:59:51.978540,/listings/17713042-walter-van-beirendonck-walt...
3,massimo osti,archive tracksuit jacket,120.0,2020-12-16 21:57:51.984344,/listings/17739704-massimo-osti-archive-90ies-...
4,nike × vintage,nike vintage puffer down jacket fleece,92.0,2020-12-16 21:56:51.986990,/listings/18182447-nike-x-vintage-nike-vintage...


In [261]:
median_price_df = clean_df.merge(cleaner_df, left_index=True, right_index = True)
median_price_df.head()

,brand1,brand2,brand3,brand,title,sold_price,when_sold,item_links
0,harrington,polo ralph lauren,vintage,harrington × polo ralph lauren × vintage,polo golf corduroy velvet check jacket,72.8,2020-12-16 22:02:51.900669,/listings/16896292-harrington-x-polo-ralph-lau...
1,ysl pour homme,yves saint laurent,nan,ysl pour homme × yves saint laurent,ysl yves saint laurent short sleeve tshirt siz...,48.0,2020-12-16 22:01:51.974796,/listings/18764205-ysl-pour-homme-x-yves-saint...
2,walter van beirendonck,nan,nan,walter van beirendonck,walter van beirendonck aw hand on heart sweat...,132.0,2020-12-16 21:59:51.978540,/listings/17713042-walter-van-beirendonck-walt...
3,massimo osti,nan,nan,massimo osti,archive tracksuit jacket,120.0,2020-12-16 21:57:51.984344,/listings/17739704-massimo-osti-archive-90ies-...
4,nike,vintage,nan,nike × vintage,nike vintage puffer down jacket fleece,92.0,2020-12-16 21:56:51.986990,/listings/18182447-nike-x-vintage-nike-vintage...


In [262]:
median_price_df.drop(['brand', 'when_sold', 'item_links'], axis = 1, inplace = True)
median_price_df

,brand1,brand2,brand3,title,sold_price
0,harrington,polo ralph lauren,vintage,polo golf corduroy velvet check jacket,72.8
1,ysl pour homme,yves saint laurent,nan,ysl yves saint laurent short sleeve tshirt siz...,48.0
2,walter van beirendonck,nan,nan,walter van beirendonck aw hand on heart sweat...,132.0
3,massimo osti,nan,nan,archive tracksuit jacket,120.0
4,nike,vintage,nan,nike vintage puffer down jacket fleece,92.0
...,...,...,...,...,...
8895,moncler,nan,nan,jacket,168.0
8896,maison margiela,nan,nan,maison margiela stereotype hoodie black,138.4
8897,burberry,vintage,nan,burberry specialty duffle coating wool coat w...,28.0
8898,prada,nan,nan,prada cotton polo shirt,68.8


Now I will run my dictionary keys over the title column and create a new column where, when there is a match with a key, the dictionary value will be inserted.  If there is no match, it will be matched with type category 17, which is our 'missing type group' category.  

In [263]:
#Thank you for the code, Flo! 
def my_dict_lookup_function(x):
    x_lst = x.split()
    for word in x_lst:
        if word in bst_category_dct.keys():
            return bst_category_dct[word]
            break
    else:
        return 17

In [264]:
median_price_df['typeid_group'] = median_price_df['title'].apply(lambda x: my_dict_lookup_function(x))
median_price_df.head()

,brand1,brand2,brand3,title,sold_price,typeid_group
0,harrington,polo ralph lauren,vintage,polo golf corduroy velvet check jacket,72.8,7
1,ysl pour homme,yves saint laurent,nan,ysl yves saint laurent short sleeve tshirt siz...,48.0,1
2,walter van beirendonck,nan,nan,walter van beirendonck aw hand on heart sweat...,132.0,1
3,massimo osti,nan,nan,archive tracksuit jacket,120.0,6
4,nike,vintage,nan,nike vintage puffer down jacket fleece,92.0,7


I want to see how many rows ended up with a "17" type group, so I know how many did not match any words from our dictionary.  I see it is almost 2,000, which means we have quite a ways to go to improve our matching percentage.  A next step will be to examine those and see if there are any patterns for what is falling through the cracks. 

In [265]:
median_price_df['typeid_group'].value_counts()

1     3170
17    1922
7     1313
3     1082
13     288
9      277
11     249
10     239
12     125
14      66
8       51
6       51
4       45
16       7
5        6
2        5
15       4
Name: typeid_group, dtype: int64

For now, however, I will build the function that will calculate the median price for any brand-type_group combination.<br><br>
In order to do that, however, I want to combine all of the brands from the 3 different brand columns into one column. This means we will have duplicates of items which have more than one brand associated to it.  We decided to do this because we believe that, even though it is rather crude, that with enough data, it will still help us get an approximate picture of what an accurate median price is for a specified brand-type_group combination.  As we develop this pipeline, we can try to make it more sophisticated and add more parameters, such as multiple brands, more specific items, fabrics, etc.

In [266]:
median_price_df2 = median_price_df.copy()
median_price_df3 = median_price_df.copy()

In [267]:
median_price_df.drop(columns=['brand2', 'brand3'], inplace=True)
median_price_df2.drop(columns=['brand1', 'brand3'], inplace=True)
median_price_df3.drop(columns=['brand1', 'brand2'], inplace=True)

In [268]:
median_price_df.rename(columns = {'brand1':'brand'}, inplace = True)
median_price_df2.rename(columns = {'brand2':'brand'}, inplace = True)
median_price_df3.rename(columns = {'brand3':'brand'}, inplace = True)

In [269]:
median_price_df.append(median_price_df2)

,brand,title,sold_price,typeid_group
0,harrington,polo golf corduroy velvet check jacket,72.8,7
1,ysl pour homme,ysl yves saint laurent short sleeve tshirt siz...,48.0,1
2,walter van beirendonck,walter van beirendonck aw hand on heart sweat...,132.0,1
3,massimo osti,archive tracksuit jacket,120.0,6
4,nike,nike vintage puffer down jacket fleece,92.0,7
...,...,...,...,...
8895,nan,jacket,168.0,7
8896,nan,maison margiela stereotype hoodie black,138.4,1
8897,vintage,burberry specialty duffle coating wool coat w...,28.0,11
8898,nan,prada cotton polo shirt,68.8,1


In [270]:
median_price_df.append(median_price_df3)

,brand,title,sold_price,typeid_group
0,harrington,polo golf corduroy velvet check jacket,72.8,7
1,ysl pour homme,ysl yves saint laurent short sleeve tshirt siz...,48.0,1
2,walter van beirendonck,walter van beirendonck aw hand on heart sweat...,132.0,1
3,massimo osti,archive tracksuit jacket,120.0,6
4,nike,nike vintage puffer down jacket fleece,92.0,7
...,...,...,...,...
8895,nan,jacket,168.0,7
8896,nan,maison margiela stereotype hoodie black,138.4,1
8897,nan,burberry specialty duffle coating wool coat w...,28.0,11
8898,nan,prada cotton polo shirt,68.8,1


I will now drop any row that has a NaN value because those are duplicates that we don't need.

In [271]:
median_price_df.dropna(inplace=True)

I now have my dataframe that will allow me to calculate the median price for a brand-type_group combination.  However, I will first turn it into a two-dimentional list so I can iterate over it, create a list of all of the prices for items sold with a specific input, and eventually return me the median price. 

In [289]:
median_price_list = median_price_df.values.tolist()

In [286]:
x = int(input("Please enter a typegroup id number: "))
y = input('Please enter your brand (all lowercase):')

Please enter a typegroup id number: 7
Please enter your brand (all lowercase):harrington


In [290]:
sold_price_list=[]
for item in median_price_list:
    if item[0]== y and item[3]==x:
        sold_price_list.append(item[2])
        
median_price = statistics.median(sold_price_list)
median_price

72.8

## For the Future: 
<ul>
    <li>investigate and improve matching dictionary to reduce items that are categorized as "missing type group" and/or are miscategorized</li>
    <li>clean python code and create as many functions as possible to automate the data processing</li>
    <li>set up pipeline from webscraping directly to postgresql database, especially for scraping from other sources</li>
    <li>investigate and develop NLP skills to set up NLP pipeline to read title texts and properly match them to business type groups</li>
    <li>track pricing and sales data to see if any pricing adjustments made, based on this data improves sales</li>
    <li>iterate, iterate, iterate!</li>